In [155]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import date
from datetime import timedelta
from datetime import datetime

%run web_scraping.ipynb
%run useful_functions.ipynb

In [145]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

In [146]:
#shows_to_run = ["https://www.bbc.co.uk/programmes/m00008zy/episodes/guide", 
#                "https://www.bbc.co.uk/programmes/b03hjfww/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b006wkh9/episodes/guide",
#                "https://www.bbc.co.uk/programmes/p04m6srg/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m000m5bw/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m0016zh5/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072lb2/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072q60/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b00p4l4t/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072l4x/episodes/guide"]

temp_show_broadcast_details = pd.DataFrame()

#for show_url in shows_to_run:
for show_url in shows['Show URL']:
    show_instance = Show(show_url, max_pages = 1)
    if shows.empty or show_instance.show_key not in shows['Show Key'].unique():
        shows = pd.concat([shows,show_instance.show_details])
    temp_show_broadcast_details = pd.concat([temp_show_broadcast_details,show_instance.broadcasts])
    
temp_show_broadcast_details['Datetime Key'] = datetime.now()
temp_show_broadcast_details['Is New Data'] = True
temp_show_broadcast_details['Is Maybe Pending'] = True
temp_show_broadcast_details.drop(columns=['Broadcast Number'], inplace=True)

show_broadcast_details = pd.concat([show_broadcast_details,temp_show_broadcast_details])
show_broadcast_details = show_broadcast_details.groupby(allbut(show_broadcast_details.columns,['Datetime Key','Is New Data','Is Maybe Pending'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
show_broadcast_details = show_broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

In [156]:
max_trials = 50
new_broadcast_data = show_broadcast_details[(show_broadcast_details["Is Broadcast Upcoming"]==False) & ((show_broadcast_details["Is New Data"]==True) | (show_broadcast_details['Is Maybe Pending'] == True))][0:max_trials]
temp_broadcast_details = pd.DataFrame()
temp_broadcast_playlists = pd.DataFrame()

for i, [broadcast_url,broadcast_key] in enumerate(new_broadcast_data[['Broadcast URL','Broadcast Key']].values):
    broadcast_instance = Playlist(broadcast_url)
    if (i+1) % 5 == 0:
        print(f'Broadcast Number {1+i} of {len(new_broadcast_data.index)}')
    if (len(broadcast_instance.playlist.index) == 0) & ((pd.to_datetime('now')- broadcast_instance.broadcast_details["Service Date"]).dt.days.values <= 7):
        show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = True
    else:
        show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = False
    temp_broadcast_details = pd.concat([temp_broadcast_details,broadcast_instance.broadcast_details])
    temp_broadcast_playlists = pd.concat([temp_broadcast_playlists,broadcast_instance.playlist])
    
temp_broadcast_details['Datetime Key'] = datetime.now()
temp_broadcast_details['Is New Data'] = True
temp_broadcast_playlists['Datetime Key'] = datetime.now()
temp_broadcast_playlists['Is New Data'] = True

broadcast_details = pd.concat([broadcast_details,temp_broadcast_details])
broadcast_details = broadcast_details.groupby(allbut(broadcast_details.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_details = broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

broadcast_playlists = pd.concat([broadcast_playlists,temp_broadcast_playlists])
broadcast_playlists = broadcast_playlists.groupby(allbut(broadcast_playlists.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_playlists = broadcast_playlists.groupby(["Broadcast Key", "Broadcast Position"]).apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)


Broadcast Number 5 of 48
Broadcast Number 10 of 48
Broadcast Number 15 of 48
Broadcast Number 20 of 48
Broadcast Number 25 of 48
Broadcast Number 30 of 48
Broadcast Number 35 of 48
Broadcast Number 40 of 48
Broadcast Number 45 of 48


In [159]:
show_broadcast_details['Is New Data'] = False
broadcast_details['Is New Data'] = False
broadcast_playlists['Is New Data'] = False

shows.to_pickle("../data/shows.pkl")
show_broadcast_details.to_pickle("../data/show_broadcast_details.pkl")
broadcast_details.to_pickle("../data/broadcast_details.pkl")
broadcast_playlists.to_pickle("../data/broadcast_playlists.pkl")

In [161]:
shows
show_broadcast_details
broadcast_details
broadcast_playlists

,Artist Image URL,Artist Name,Track Name,Record Label,Track Number,Broadcast Position,Broadcast Key,Datetime Key,Is New Data
0,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Crawlers,Didn't Know How To Say,,,1.0,-9078513893279848143,2022-12-11 13:43:05.300076,False
1,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,RedHook,Jabberwocky,,,2.0,-9078513893279848143,2022-12-11 13:43:05.300076,False
2,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,LIA LIA,2young2die,,,3.0,-9078513893279848143,2022-12-11 13:43:05.300076,False
3,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Crystal Tides,Sorry,,,4.0,-9078513893279848143,2022-12-11 13:43:05.300076,False
4,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,FES,Sun Visor,,,5.0,-9078513893279848143,2022-12-11 13:43:05.300076,False
...,...,...,...,...,...,...,...,...,...
4452,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Ólafur Arnalds,Near Light,,,42.0,9176309020129642538,2022-12-11 13:47:18.251800,False
4453,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Peter Gabriel,The Nest That Sailed the Sky,,,43.0,9176309020129642538,2022-12-11 13:47:18.251800,False
4454,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Above & Beyond,Small Moments,,,44.0,9176309020129642538,2022-12-11 13:47:18.251800,False
4455,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Above & Beyond,Sun In Your Eyes,,,45.0,9176309020129642538,2022-12-11 13:47:18.251800,False
